In [1]:
! pip install pywikibot
! pip install mwparserfromhell
! pip install watchdog

     |████████████████████████████████| 522kB 587kB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/6c/07/84/ba6645f74fe204adce4ec9a49365afdc5c526f1250691c5e41
Successfully built pywikibot
     |████████████████████████████████| 143kB 638kB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/2a/76/d5/7088b941df3b362c45dd7912dd05314bc034751ec9cbca9a75
Successfully built mwparserfromhell
     |████████████████████████████████| 92kB 867kB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31c42adb4e42b76495bc8f784c
  Stored in directory: /home/jovyan/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built watchdog pathtools


In [2]:
import os

os.environ['PYWIKIBOT_DIR'] = './wiki_reader/'

In [3]:
# import sparknlp
# from pyspark.sql import SQLContext

# spark = sparknlp.start()
# sc = spark.sparkContext
# sqlCtx = SQLContext(sc)

In [4]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType, DoubleType

from spark_app.scorers import score_text
from spark_app.spark_tools import SparkSentimentStreamer
from pathlib import Path

## Input

In [5]:
request = 'peano'

batch_size = 100
limit = None
preload_content = True
is_category = False

wiki_page_dir = 'requests/'
spark_results_dir = 'responses/'

## Processing

In [6]:
# from ml.LogisticRegressionCached import readFromCache

# (lrModel, pipelineModel) = readFromCache('./ml/train/')

# def score_text_ml(text):
#     df = spark.createDataFrame([(text, 2)], ['text', 'target'])
#     df_transformed = pipelineModel.transform(df) # To fix
#     predictions = lrModel.transform(df_transformed)
#     predictions = predictions.select(['text', 'probability', 'prediction'])
#     pd_predictions = predictions.toPandas()
#     positive_probability = pd_predictions.iloc[0]['probability'][1]
#     overall_probability = 2 * positive_probability - 1
    
#     return overall_probability

In [7]:
from ml.OutputProcessing import process_output_files

# process_output_files('./responses/', 1000)

In [8]:
def spark_process(request, score_func):
    spark = SparkSession.builder\
        .master("local[*]")\
        .appName("NetworkWordCount")\
        .getOrCreate()
    
    sc = spark.sparkContext
    ssc = StreamingContext(sc, 1)
    
    dataInp = wiki_page_dir + request
    dataOut = spark_results_dir + request
    Path(dataOut).mkdir(parents=True, exist_ok=True)
    
    streamer = SparkSentimentStreamer(sc, ssc, spark, score_func, dataInp, dataOut)
    streamer.run()
    streamer.stop()

In [9]:
from concurrent.futures import ThreadPoolExecutor
import wiki_reader.reader as reader

wiki_wrapper = lambda r,b,l,cat,cont: reader.query(r,out_dir=wiki_page_dir,batch_size=b,
                                              limit=l,is_category=cat,preload_content=cont)

with ThreadPoolExecutor(max_workers=2) as e:
    e.submit(wiki_wrapper, request, batch_size, limit, is_category, preload_content)
    e.submit(spark_process, request, score_text)
    # e.submit(process_output_files, './responses/', 1000)

Cleaning old data
Downloading...
Dumped 100 pages | peano
Dumped 200 pages | peano
+--------------------+--------------------+--------------------+
|                text|               title|                 url|
+--------------------+--------------------+--------------------+
|0 (zero) is both ...|                   0|https://en.wikipe...|
|1 (one, also call...|                   1|https://en.wikipe...|
|thumb|right| Flow...|           Algorithm|https://en.wikipe...|
|thumb|right|Giuse...|Arithmetices prin...|https://en.wikipe...|
|In mathematics, a...|    Axiomatic system|https://en.wikipe...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

Dumped 300 pages | peano
Dumped 400 pages | peano
+--------------------+--------------------+--------------------+
|                text|               title|                 url|
+--------------------+--------------------+--------------------+
|In mathematics, a...| Algebraic structure|https://en.wikipe..